# SyftBox SDK Development Testing

This notebook is for testing and validating the SyftBox SDK during development.

In [1]:
# Import the SDK
import syft_hub as sh
import logging

In [2]:
# Import client
from syft_hub import Client
from syft_hub.utils.logger import get_logger

In [3]:
# define utilities
logger = get_logger()
logging.getLogger('syft_hub').setLevel(logging.INFO)
logger.success("Logger initialized successfully!")

2025-10-03 00:42:03 - syft_hub - SUCCESS - Logger initialized successfully!


In [4]:
# Initialize client
client = Client()

2025-10-03 00:42:04 - syft_hub.main - INFO - Connected to existing accounting account for callis@openmined.org
2025-10-03 00:42:04 - syft_hub.main - INFO - Client initialized for callis@openmined.org


In [5]:
# search service search_async
result = await client.search(
    service_name="callis@openmined.org/carl-model",
    message="What are AI agentic systems?",
    topK=5,
    similarity_threshold=0.8
)

2025-10-03 00:42:04 - syft_hub.main - INFO - Using service: carl-model from datasite: callis@openmined.org


✓ Response received                                                             



In [6]:
# show response
logger.info(f"Search result:\n\n {result}")
# logger.info(f"Search result:\n\n {client.format_search_context(result, "simple")}")

2025-10-03 00:42:09 - syft_hub - INFO - Search result:

 Search results for: 'What are AI agentic systems?'

1. Score: 0.544
   er (Wang et al., 2023b) is a good example for agents that use feedback from the simulated environmen...

2. Score: 0.539
   e agent interacts with the agricultural environment by taking actions in agricultural management and...

3. Score: 0.523
   in the construction of LLM-based intelligent agents. As shown in §6, we showed three major scenarios...

4. Score: 0.522
   media in the construction of agents?

A lot of work has begun to adopt this approach, like Voyager (...

5. Score: 0.517
   t https://github.com/ morecry/CharacterEval .

## 1 Introduction

The development of large language ...
